In [39]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import bs4 as bs

In [40]:
#get an url list 
def get_url_list(page1_url):
    var_url = '&section_offset=5&items_offset='
    url_lst = [page1_url]
    for page in range(1,17):
        url = page1_url+var_url+str(18*page)
        url_lst.append(url)
    return url_lst

In [41]:

def get_house_info(url, price):
    #url = 'https://www.airbnb.com/rooms/3503946?s=P8Cyps_7'
    import requests
    from bs4 import BeautifulSoup
    import re
    response = requests.get(url)
    output = []
    if not response.status_code == 200:
        print('failed to get the fidelity website')
        return output
    else:
        results_page = BeautifulSoup(response.content,'html5lib')
        # rating
        if len(results_page.find_all('div', class_="_l0ao8q")) == 0:
            rating = 'no rating'
        else:
            rating_sec = results_page.find_all('div', class_="_l0ao8q")[0]
            rating = rating_sec.find('span').get('aria-label')
        # coordinates
        coordinates = re.search(
            '"listing_lat":([0-9]{2}.[0-9]*),"listing_lng":(-[0-9]{2}.[0-9]*),', 
            response.text)
        
        lat = coordinates.group(1)
        lon = coordinates.group(2)
        # host
        superhost = re.search('"is_superhost":(.{1,5})', 
                              response.text).group(1)
        
        if len(results_page.find_all('span', class_="_fgdupie"))==0:
            response_rate = "no info"
        else:
            host = results_page.find_all('span', class_="_fgdupie")
            #response rate
            response_rate = float('nan')
            for host_ in host[::-1]:
                if 'Response rate' in host_.text:
                    response_rate = host_.text.split(':')[-1]
                    break
                
      
        if re.search('"guest_label":".{1,8}([0-9][0-9]?).{1,8}",', response.text) != None:
            accommodates = re.search('"guest_label":".{1,8}([0-9][0-9]?).{1,8}",', response.text).group(1)
        else:
            accommodates = re.search('"guest_label":"([0-9][0-9]?) guest.*', response.text).group(1)
        #print (accommodates)
        
        
        #bedrooms
        if re.search('"bedroom_label":"([0-9][0-9]?).*","guest_label"', response.text) != None:
            bedrooms = (re.search('"bedroom_label":"([0-9][0-9]?).*","guest_label"', response.text)).group(1)
        else:
            bedrooms = "studio"
            #print (bedrooms)
            
        #bathrooms
        bathrooms = (re.search('"bathroom_label":"([0-9]\.?[0-9]?).*","bed_label"', response.text)).group(1)
        
        #beds
        beds = (re.search('"bed_label":"(.).*","bedroom_label"', response.text)).group(1)
        #print (beds)
        
        #room type
        try:
            room_type = results_page.find('span',class_="_1hh2h7tb").text
        except AttributeError:
            room_type='NoneType'

        #number of review
        try:
            number_of_reviews = results_page.find('div', class_="_vy3ibx").text
        except AttributeError:
            number_of_reviews='NoneType'
        
        output=[price, accommodates, bedrooms,beds, bathrooms, room_type, number_of_reviews,  rating, lat, lon, superhost, response_rate]
        #print('output=')
        #print(output)
        
    return output

In [42]:
def get_prices_urls(url,dataframe):
    info_list=[]
    from selenium import webdriver  
    browser = webdriver.Chrome('/Users/daqiliu/Desktop/chromedriver')
    #url='https://www.airbnb.com/s/Manhattan--New-York--NY--United-States/homes?refinement_paths%5B%5D=%2Fhomes&toddlers=0&query=Manhattan%2C%20New%20York%2C%20NY%2C%20United%20States&allow_override%5B%5D=&price_min=100&price_max=199&s_tag=qfon6p2A'
    browser.get(url)
    html_source = browser.page_source
    #browser.quit()
    #get the prices for every apartment since the price infomation is not availabe inside the apt page
    prices = browser.find_elements(By.XPATH,"//span[@class='_17oldnte']")
    price_list = []
    for price in prices:
        try:
            price = price.text
            price_list.append(price)
        except:
            pass
    price_list = price_list[::3]
    browser.quit()
    print("length of price list is: ", len(price_list))

    #get the url for every apt on this page
    results_page = BeautifulSoup(html_source, "lxml")
    urls = results_page.find_all("meta", content=re.compile("www.airbnb.com/rooms"))
    #urls = browser.find_elements(By.XPATH,"//a[@class='_1ol0z3h']")
    url_list = []
    for url in urls:
        url = str(url)
        url = "https://"+url[15:-18]
        #print (x)
        url_list.append(url)
    print("length of apt_url_list is:" + str(len(url_list)))
    print("-----------------")
    #print(url_list)
    #pass url and price to the get_house_info function and extract the rest infomation 
    
    if (len(price_list) == len(url_list)) and (len(price_list)>0):
        for x in range(len(price_list)):
            #print(get_house_info(url_list[x], price_list[x]))
            info_list.append(get_house_info(url_list[x], price_list[x]))
        #print('info list is:')
        #print(info_list)
        df2=pd.DataFrame(info_list,columns=['price','accommodates', 'bedrooms','beds', 
                                            'bathrooms', 'room_type', 'number_of_reviews',  'rating', 
                                            'lat', 'lon', 'superhost', 'response_rate'])
        #print('dataframe is:')
        #print(df2)
        
        dataframe=pd.concat([dataframe,df2], ignore_index=True)
        print('complete dataframe')

    #print("info_list=")
    #print(info_list)
    return dataframe  

In [43]:
page_url="https://www.airbnb.com/s/Manhattan--New-York--NY--United-States/homes?refinement_paths%5B%5D=%2Fhomes&checkin=2019-02-14&checkout=2019-02-28&adults=0&children=0&infants=0&guests=0&toddlers=0&query=Manhattan%2C%20New%20York%2C%20NY%2C%20United%20States&price_min=27&price_max=43&allow_override%5B%5D=&s_tag=izDMX3FF"
url_list=get_url_list(page_url)
df = pd.DataFrame(columns=['price','accommodates', 'bedrooms','beds', 'bathrooms', 'room_type', 'number_of_reviews',  'rating', 'lat', 'lon', 'superhost', 'response_rate'])
for url in url_list:
    #create a df   
    df=get_prices_urls(url,df)

In [44]:
df

,price,accommodates,bedrooms,beds,bathrooms,room_type,number_of_reviews,rating,lat,lon,superhost,response_rate
0,Price$34,1,1,0,1,Private room in apartment,NoneType,no rating,40.81209932511971,-73.95934462235282,false,NaN
1,Price$35,2,1,1,1,Private room in apartment,14 Reviews,Rating 5 out of 5,40.85133080947567,-73.94006003288479,false,100%
2,Price$41,1,1,1,1,Private room in apartment,59 Reviews,Rating 5 out of 5,40.829571471763984,-73.94534744481334,false,100%
3,Price$40,3,1,1,2,Private room in apartment,20 Reviews,Rating 5 out of 5,40.82815763393119,-73.94957162323941,false,100%
4,Price$43,2,1,1,1,Private room in apartment,55 Reviews,Rating 4.5 out of 5,40.82743813688141,-73.94706710208679,false,100%
5,Price$41,2,1,1,1,NoneType,NoneType,no rating,40.799778403682936,-73.94142972122107,false,no info
6,Price$39,2,1,1,1.5,Private room in apartment,6 Reviews,Rating 4.5 out of 5,40.81850343734161,-73.95622089720574,false,100%
7,Price$38,2,1,1,1,Private room in apartment,55 Reviews,Rating 5 out of 5,40.838776424912965,-73.94731391112843,"true,",NaN
8,Price$36,2,1,1,1,Private room in apartment,4 Reviews,Rating 5 out of 5,40.86764244785547,-73.92590186542682,false,NaN
9,Price$30,2,1,1,1,Private room in apartment,1 Review,no rating,40.853589279863485,-73.92982059524118,false,NaN


In [45]:
df.to_csv('/Users/daqiliu/Desktop/airbnbdata.csv')